<a href="https://colab.research.google.com/github/georgestein/multi_label_pigeon/blob/master/pidgeon_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/georgestein/multi_label_pigeon.git

  Cloning https://github.com/georgestein/multi_label_pigeon.git to /tmp/pip-req-build-ga7b8kn7
  Running command git clone -q https://github.com/georgestein/multi_label_pigeon.git /tmp/pip-req-build-ga7b8kn7
  Created wheel for multi-label-pigeon: filename=multi_label_pigeon-0.3.0-cp37-none-any.whl size=5252 sha256=f41d0d4d8cfc955c711b982a6419b4b8ade61fc3373b92254aeda07424f3f6c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-1os1m6so/wheels/dc/2b/41/bdad1c0f0c71ede872aaad1fcd5f9ca48dee5d12d109b473f9
Successfully built multi-label-pigeon


In [2]:
!wget https://portal.nersc.gov/project/nyx/decals_self_supervised/lens_images/lens_trial.npy

--2021-04-28 00:00:34--  https://portal.nersc.gov/project/nyx/decals_self_supervised/lens_images/lens_trial.npy
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.28, 128.55.206.24
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62935424 (60M)
Saving to: ‘lens_trial.npy’

lens_trial.npy      100%[===================>]  60.02M  21.5MB/s    in 2.8s    

2021-04-28 00:00:37 (21.5 MB/s) - ‘lens_trial.npy’ saved [62935424/62935424]



In [3]:
# dr8 to images
# https://github.com/legacysurvey/imagine/blob/master/map/views.py
def sdss_rgb(imgs, bands, scales=None,
             m = 0.02):
    import numpy as np
    rgbscales = {'u': (2,1.5), #1.0,
                 'g': (2,2.5),
                 'r': (1,1.5),
                 'i': (0,1.0),
                 'z': (0,0.4), #0.3
                 }
    if scales is not None:
        rgbscales.update(scales)

    I = 0
    for img,band in zip(imgs, bands):
        plane,scale = rgbscales[band]
        img = np.maximum(0, img * scale + m)
        I = I + img
    I /= len(bands)
        
    # b,g,r = [rimg * rgbscales[b] for rimg,b in zip(imgs, bands)]
    # r = np.maximum(0, r + m)
    # g = np.maximum(0, g + m)
    # b = np.maximum(0, b + m)
    # I = (r+g+b)/3.
    Q = 20
    fI = np.arcsinh(Q * I) / np.sqrt(Q)
    I += (I == 0.) * 1e-6
    H,W = I.shape
    rgb = np.zeros((H,W,3), np.float32)
    for img,band in zip(imgs, bands):
        plane,scale = rgbscales[band]
        rgb[:,:,plane] = (img * scale + m) * fI / I

    # R = fI * r / I
    # G = fI * g / I
    # B = fI * b / I
    # # maxrgb = reduce(np.maximum, [R,G,B])
    # # J = (maxrgb > 1.)
    # # R[J] = R[J]/maxrgb[J]
    # # G[J] = G[J]/maxrgb[J]
    # # B[J] = B[J]/maxrgb[J]
    # rgb = np.dstack((R,G,B))
    rgb = np.clip(rgb, 0, 1)
    return rgb

def dr2_rgb(rimgs, bands, **ignored):
    return sdss_rgb(rimgs, bands, scales=dict(g=(2,6.0), r=(1,3.4), z=(0,2.2)), m=0.03)

def _unwise_to_rgb(imgs, bands=[1,2],
                   scale1=1.,
                   scale2=1.,
                   arcsinh=1./20.,
                   mn=-20.,
                   mx=10000., 
                   w1weight=9.):
    import numpy as np
    img = imgs[0]
    H,W = img.shape

    ## FIXME
    assert(bands == [1,2])
    w1,w2 = imgs
    
    rgb = np.zeros((H, W, 3), np.uint8)

    # Old:
    # scale1 = 50.
    # scale2 = 50.
    # mn,mx = -1.,100.
    # arcsinh = 1.

    img1 = w1 / scale1
    img2 = w2 / scale2

    if arcsinh is not None:
        def nlmap(x):
            return np.arcsinh(x * arcsinh) / np.sqrt(arcsinh)

        # intensity -- weight W1 more
        bright = (w1weight * img1 + img2) / (w1weight + 1.)
        I = nlmap(bright)

        # color -- abs here prevents weird effects when, eg, W1>0 and W2<0.
        mean = np.maximum(1e-6, (np.abs(img1)+np.abs(img2))/2.)
        img1 = np.abs(img1)/mean * I
        img2 = np.abs(img2)/mean * I

        mn = nlmap(mn)
        mx = nlmap(mx)

    img1 = (img1 - mn) / (mx - mn)
    img2 = (img2 - mn) / (mx - mn)

    rgb[:,:,2] = (np.clip(img1, 0., 1.) * 255).astype(np.uint8)
    rgb[:,:,0] = (np.clip(img2, 0., 1.) * 255).astype(np.uint8)
    rgb[:,:,1] = rgb[:,:,0]/2 + rgb[:,:,2]/2

    return rgb

In [4]:
import numpy as np
from multi_label_pigeon import multi_label_annotate
from IPython.display import display, Image
import matplotlib.pyplot as plt
from google.colab import files

def display_gal(imi, npix=152):

    pix_start = imi.shape[-1]//2 - npix//2
    pix_end = imi.shape[-1]//2 + npix//2

    fig = plt.figure(figsize=(10,10))
    # im = images[fi]

    rgb = dr2_rgb(imi[:, pix_start:pix_end, pix_start:pix_end], ['g','r','z'])
    plt.imshow(rgb, interpolation='none')

    # plt.imshow(imi)
    plt.show(block=False)
    plt.pause(3)
    plt.close()


In [10]:
lens_images = np.load('lens_trial.npy')
ims_label = np.arange(lens_images.shape[0])

# shuffle here, so that can manually restart labelling if crashes in middle
np.random.seed(42)
np.random.shuffle(ims_label)
lens_images = lens_images[ims_label]

# start from start_image and show till last
# change if desired
start_image = 0

save_file = 'annotations_{:04d}'.format(start_image)

lens_options = {'Lens?': ['yes', 'no', 'possibly'],
                'Arc?': ['small', 'big', 'none'],
                'Multiple sources?': ['yes', 'no', 'maybe'],
                'Number of galaxies?': ['1', '2', '3', '4+'],
                'Radius from central galaxy?': ['small', 'medium', 'large'],
                'Anomaly?': ['yes', 'no']
                }


annotations = multi_label_annotate(
    lens_images[start_image:], example_labels=ims_label[start_image:],
    options=lens_options,
    display_fn=display_gal,
    save_file=save_file,
    shuffle=False)

# RUN NEXT 2 CELLS TO SAVE FINAL CATALOGUE AND DOWNLOAD

HTML(value='0 examples annotated, 228 examples left<br>Asset name: <code>102</code><hr>')

Lens?


Arc?


Multiple sources?


Number of galaxies?


Radius from central galaxy?


Anomaly?


Output()

In [6]:
# See all files you have on disk
!ls 

annotations_00002.npy  lens_trial.npy  sample_data


In [7]:
# If you saved an intermediate catalogue then download it
f = 'annotations_00002.npy'
files.download(f) 


FileNotFoundError: ignored

In [8]:
# else you finished, and want to download all labels
save_file_all = 'annotations_all.npy'
np.save(save_file_all, annotations)

# download final dictionary of labels
files.download(save_file_all) 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
annotations

defaultdict(dict, {1: {'Arc?': ['small'], 'Lens?': ['yes']}})